# Chapter 16 - Natural Language Processing with RNNs and Attention

In [1]:
# Common imports
import numpy as np
import os

import sklearn
import tensorflow as tf
from tensorflow import keras

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

2022-05-24 11:17:03.835937: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-24 11:17:03.835969: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Generating Shakesperian Text Using a Character RNN

Let's build a Char-RNN, an RNN whose task is to predict the next character in a sentence. We'll train the RNN using some sentences from Shakespeare.

In [2]:
shakespeare_url = "https://homl.info/shakespeare" # shortcut url
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

Next, we must encode every character as an integer. One option is to create a custom preprocessing layer, as we did in Chapter 13. But in this case, it will be simpler to use Keras’s `Tokenizer` class. First we need to fit a tokenizer to the text: it will find all the characters used in the text and map each of them to a different character ID, from 1 to the number of distinct characters (it does not start at 0, so we can use that value for masking, as we will see later in this chapter):

In [3]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

In [4]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [5]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [6]:
# number of distinct characters
max_id = len(tokenizer.word_index)
max_id

39

In [7]:
# total number of characters
dataset_size = tokenizer.document_count
dataset_size

1

Let's encode the full text so each character is represented by its ID (we subtract 1 to get IDs from 0 to 38, rather than from 1 to 39):

In [8]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

To split the dataset into a training set, a validation set, and a test set, we can't simply shuffle it, since we are dealing with sequential data. 

### How to Split a Sequential Dataset

It is very important to avoid any overlap between the training set, the validation set, and the test set. For example, we can take the first 90% of the text for the training set, then the next 5% for the validation set, and the final 5% for the test set. It would also be a good idea to leave a gap between these sets to avoid the risk of a paragraph overlapping over two sets.

In [9]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

2022-05-24 11:17:46.311854: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-24 11:17:46.311888: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-24 11:17:46.311921: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (alexandre-Inspiron-5458): /proc/driver/nvidia/version does not exist
2022-05-24 11:17:46.530533: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Chopping the Sequential Dataset Into Multiple Windows

The training set now consists of a single sequence of over a million characters, so we can’t just train the neural network directly on it: the RNN would be equivalent to a deep net with over a million layers, and we would have a single (very long) instance to train it. Instead, we will use the dataset's `window()` method to convert this long sequence of characters into many smaller windows of text. Every instance in the dataset will be a fairly short substring of the whole text, and the RNN will be unrolled only over the length of these substrings. This is called *truncated backpropagation through time*. 

In [10]:
n_steps = 100
window_length = n_steps + 1  # target = input shifted 1 character ahead
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

The `window()` function returns a `nested dataset`, analogous to a list of lists. However, the `fit` method only accepts tensors as input. Therefore, we must use the function method `flat_map` to flatten that list of datasets. Note that we can also pass a function to `flat_map` to perform transform operations before flattening.

In [11]:
# since all windows have exactly that length, we will get a single tensor for each of them
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [12]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In summary, so far we have performed the following operations:

![pipeline_data_transformation](./images/ch16_pipeline_sequential_data_transformation.png)

In [13]:
# encoding the characters
dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

# prefetching
dataset = dataset.prefetch(1)

### Building and Training the Char-RNN Model

In [27]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=1)

### Using the Char-RNN Model

To feed new data to the model, we need to preprocess it.

In [14]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [ ]:
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]  # 1st sentence, last char

### Generating Fake Shakesperian Text

To generate new text using the Char-RNN model, we could feed it some text,
make the model predict the most likely next letter, add it at the end of the text,
then give the extended text to the model to guess the next letter, and so on. But
in practice this often leads to the same words being repeated over and over
again. Instead, we can pick the next character randomly, with a probability equal
to the estimated probability, using TensorFlow’s `tf.random.categorical()`
function. This will generate more diverse and interesting text. The
`categorical()` function samples random class indices, given the class log
probabilities (logits). To have more control over the diversity of the generated
text, we can divide the logits by a number called the `temperature`, which we can
tweak as we wish: a temperature close to 0 will favor the high-probability
characters, while a very high temperature will give all characters an equal
probability. The following `next_char()` function uses this approach to pick the
next character to add to the input text:

In [15]:
def next_char(text, temperature=1):
    X_new = preprocess(text)
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

Next, we can write a small function that will repeatedly call `next_char()` to get
the next character and append it to the given text:

In [16]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

Let's text some inputs:

In [17]:
# print(complete_text("t", temperature=0.2))

In [18]:
# print(complete_text("w", temperature=1))

In [19]:
# print(complete_text("w", temperature=2))

### Stateful RNN

Until now, we have used only *stateless* RNNs: at each training iteration the
model starts with a hidden state full of zeros, then it updates this state at each
time step, and after the last time step, it throws it away, as it is not needed
anymore. What if we told the RNN to preserve this final state after processing
one training batch and use it as the initial state for the next training batch? This
way the model can learn long-term patterns despite only backpropagating
through short sequences. This is called a *stateful* RNN. 

First, note that a stateful RNN only makes sense if each input sequence in a
batch starts exactly where the corresponding sequence in the previous batch left
off. So the first thing we need to do to build a stateful RNN is to use sequential
and nonoverlapping input sequences (rather than the shuffled and overlapping
sequences we used to train stateless RNNs).

In [20]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

![pipeline_stateful_rnn](./images/ch16_pipeline_stateful_rnn.png)

Now let's create the stateful RNN:

- Set `stateful=True`
- The RNN needs to know the batch size (since it will preserve a state for each input sequence in the batch), so we must set `batch_input_shape` in the first layer.

In [21]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2,
                     batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True, 
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

We need to reset the states at the end of each epoch, to do this, we can use the following callback:

In [22]:
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [24]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])

After this model is trained, it will only be possible to use it to make predictions for batches of
the same size as were used during training. To avoid this restriction, create an identical
stateless model, and copy the stateful model’s weights to this model.

In [25]:
stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

# Build the model
stateless_model.build(tf.TensorShape([None, None, max_id]))
stateless_model.set_weights(model.get_weights())
model = stateless_model

tf.random.set_seed(42)

print(complete_text("t"))

tdf3zwvcik :ma!&q. :phgr&;ubltcpzhp:'rv:cq3z!$ pau:


## Sentiment Analysis

In [26]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()
X_train[0][:10]

17473536/17464789 [==============================] - 7s 0us/step


[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

The reviews have already been preprocessed for us: each of which is
represented as a NumPy array of integers, where each integer represents a word.
All punctuation was removed, and then words were converted to lowercase, split
by spaces, and finally indexed by frequency (so low integers correspond to
frequent words). The integers 0, 1, and 2 are special: they represent the padding
token, the start-of-sequence (SSS) token, and unknown words, respectively

In [27]:
# decode a review
word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
    id_to_word[id_] = token
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

1654784/1641221 [==============================] - 1s 1us/step


'<sos> this film was just brilliant casting location scenery story'

If we want to deploy the model to a mobile device or a web application, we may have to handle preprocessing only using TensorFlow operations, so it can be included in the model itself. Let's see how.

In [28]:
# Load the original IMDb dataset 
import tensorflow_datasets as tfds

datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

In [29]:
datasets.keys()

dict_keys(['train', 'test', 'unsupervised'])

In [30]:
train_size = info.splits["train"].num_examples
test_size = info.splits["test"].num_examples

In [31]:
train_size, test_size

(25000, 25000)

In [32]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, rb"<br\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch

Next, we need to construct the vocabulary. This requires going through the
whole training set once, applying our `preprocess()` function, and using a
Counter to count the number of occurrences of each word

In [33]:
from collections import Counter

vocabulary = Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [44]:
vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [45]:
len(vocabulary)

53893

We probably don't need out model to know all the words in the dictionary, so let's keep only the 10000 most common words.

In [46]:
vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]
]

Now we need to replace each word with its ID (i.e., its index in the vocabulary).

In [47]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [50]:
table.lookup(tf.constant([b"This movie was faaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10053]])>

Now we are ready to create the final training set. We batch the reviews, then
convert them to short sequences of words using the `preprocess()` function,
then encode these words using a `simple encode_words()` function that uses the
table we just built, and finally prefetch the next batch:

In [51]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets["train"].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [52]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 128s 159ms/step - loss: 0.6151 - accuracy: 0.6264
Epoch 2/5
782/782 [==============================] - 102s 131ms/step - loss: 0.3842 - accuracy: 0.8332
Epoch 3/5
782/782 [==============================] - 113s 144ms/step - loss: 0.2245 - accuracy: 0.9171
Epoch 4/5
782/782 [==============================] - 109s 140ms/step - loss: 0.1448 - accuracy: 0.9493
Epoch 5/5
782/782 [==============================] - 123s 158ms/step - loss: 0.1215 - accuracy: 0.9579


### Masking

As it stands, the model will need to learn that the padding tokens should be
ignored. But we already know that! Why don’t we tell the model to ignore the
padding tokens, so that it can focus on the data that actually matters? It’s actually
trivial: simply add `mask_zero=True` when creating the Embedding layer. This
means that padding tokens (whose ID is 0) will be ignored by all downstream
layers. That’s all!

In [53]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                           mask_zero=True, # not shown in the book
                           input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 130s 156ms/step - loss: 0.5389 - accuracy: 0.7154
Epoch 2/5
782/782 [==============================] - 123s 157ms/step - loss: 0.3348 - accuracy: 0.8600
Epoch 3/5
782/782 [==============================] - 129s 165ms/step - loss: 0.1849 - accuracy: 0.9337
Epoch 4/5
782/782 [==============================] - 133s 170ms/step - loss: 0.1298 - accuracy: 0.9551
Epoch 5/5
782/782 [==============================] - 125s 160ms/step - loss: 0.1178 - accuracy: 0.9560


### Reusing Pretrained Embeddings

In [55]:
import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1",
                   dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dense(1, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

2022-05-24 12:22:26.846339: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 192762400 exceeds 10% of free system memory.


The `hub.KerasLayer` layer downloads the module from the given URL. This
particular module is a sentence encoder: it takes strings as input and encodes
each one as a single vector (in this case, a 50-dimensional vector). Internally, it
parses the string (splitting words on spaces) and embeds each word using an
embedding matrix that was pretrained on a huge corpus: the Google News 7B
corpus (seven billion words long!). Then it computes the mean of all the word
embeddings, and the result is the sentence embedding. We can then add two
simple Dense layers to create a good sentiment analysis model. By default, a
hub.KerasLayer is not trainable, but you can set `trainable=True` when
creating it to change that so that you can fine-tune it for your task

In [56]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size = info.splits["train"].num_examples
batch_size = 32
train_set = datasets["train"].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 6s 6ms/step - loss: 0.5473 - accuracy: 0.7266
Epoch 2/5
782/782 [==============================] - 4s 5ms/step - loss: 0.5143 - accuracy: 0.7484
Epoch 3/5
782/782 [==============================] - 4s 5ms/step - loss: 0.5092 - accuracy: 0.7510
Epoch 4/5
782/782 [==============================] - 4s 5ms/step - loss: 0.5058 - accuracy: 0.7538
Epoch 5/5
782/782 [==============================] - 4s 5ms/step - loss: 0.5031 - accuracy: 0.7548


## An Encoder-Decoder Network for Neural Machine Translation